In [1]:
# Mateusz Biegański
# mb385162

# SAD 2020-2021 MIMUW

In [2]:
# pretty print string formatting
library("glue");

In [7]:
# Helper functions.

m  = function(x) print(methods(class=class(x)))
p  = function(x) print(glue(x))
ph = function(x) print(head(x))
pt = function(x) print(tail(x))
pc = function(x) print(class(x))
id = function(x) x


In [41]:
# Read the data.
DATA_FNAME = "people.tsv"

df <- read.table(DATA_FNAME, header = TRUE,  sep = '\t',  stringsAsFactors = FALSE)
df[which(df$pet == "none"),]$pet = NA